# Pytorch
## Tensor
1. scalar: 0维，数字
2. vector: 1维
3. matrix: 2维
4. tensor: 3维或更多

为了简便起见，我们把vector和matrix也叫做tensor.  
tensor()既能创建scalar也能创建tensor. Pytorch里的tensor和numpy很像。

In [2]:
import torch
scalar = torch.tensor(3.14159)
vector = torch.tensor([1,2,3])
matrix = torch.ones((2,3), dtype=torch.float)
tensor = torch.randn((2,3,4), dtype=torch.float)
print(tensor)
print(tensor.size(), tensor.shape)
print(vector.size(), vector.shape)
print(scalar.size(), scalar.shape)

tensor([[[ 1.8800,  0.7844,  0.2366,  0.1843],
         [ 0.7413,  0.6363,  0.7394,  0.8137],
         [-1.0529,  0.6909,  0.4633,  0.4748]],

        [[-1.1020,  0.2785,  1.6564,  1.3391],
         [ 1.1269,  0.1828, -0.3148, -0.3454],
         [-0.2529,  0.1099,  1.3951, -1.2532]]])
torch.Size([2, 3, 4]) torch.Size([2, 3, 4])
torch.Size([3]) torch.Size([3])
torch.Size([]) torch.Size([])


In [5]:
same_matrix = matrix.view(1,6)
print(same_matrix)
same_matrix[0,1]=2
print(same_matrix)
another_matrix = same_matrix.clone().detach() # duplicate the data
print(another_matrix)

tensor([[1., 2., 1., 1., 1., 1.]])
tensor([[1., 2., 1., 1., 1., 1.]])
tensor([[1., 2., 1., 1., 1., 1.]])


## CUDA
我们可以用torch.as_tensor()很方便地把numpy数组转成tensor.

In [4]:
import numpy as np
dummy_array = np.array([1,2,3])
dummy_tensor = torch.as_tensor(dummy_array)
dummy_array[1]=0
dummy_tensor
# 注意这里是改引用而不是改拷贝。这也是我们不用torch.tensor()的原因

tensor([1, 0, 3])

以上我们只是创建了CPU tensor，接下来我们使用GPU.显然GPU可以加速模型训练。  

In [7]:
# 查看GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

# 把cpu tensor转成gpu tensor
gpu_tensor = torch.as_tensor(dummy_array).to(device)
gpu_tensor[0]

cuda
NVIDIA GeForce GTX 1060 6GB


tensor(1, device='cuda:0')

In [8]:
# Synthetic Data Generation
true_b = 1
true_w = 2
N = 100
np.random.seed(42)

x = np.random.rand(N,1)
epsilon = .1 * np.random.randn(N,1)
y = true_b + true_w * x + epsilon

# Split the synthetic data into train and validation sets.
idx = np.arange(N)
np.random.shuffle(idx)
train_idx = idx[:int(N*.8)]
val_idx = idx[int(N*.8):]
x_train,y_train = x[train_idx],y[train_idx]
x_val,y_val = x[val_idx,],y[val_idx]

In [9]:
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)
print(type(x_train), type(x_train_tensor), x_train_tensor.type())
# 用tensor_val.type()可以告诉你这是GPU tenso=1

<class 'numpy.ndarray'> <class 'torch.Tensor'> torch.cuda.FloatTensor


In [10]:
# step0 - initialize parameters
# 变量在创建时就指定好device
torch.manual_seed(42)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(b,w)

tensor([0.1940], device='cuda:0', requires_grad=True) tensor([0.1391], device='cuda:0', requires_grad=True)
